In [3]:
    from __future__ import absolute_import, division, print_function

    import tensorflow as tf
    import numpy as np

    tf.logging.set_verbosity(tf.logging.INFO)
    #Sets the threshold for what messages will be logged. message level: 'FATAL','ERROR','WARN','INFO','DEBUG'

In [4]:
PATH = "D:\深圳\大岩\项目\intern-2018\CNN\estimator_mnist"

In [5]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
  # for black-and-white photo with 28 * 28 pixels

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
  }
  #tf.summary.scalar('accuracy', accuracy[1])
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [6]:
# Load training and eval data
((train_data, train_labels),
 (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required

In [7]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir=PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'D:\\深圳\\大岩\\项目\\intern-2018\\CNN\\estimator_mnist', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E53D27B0B8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [9]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=1,
    hooks=[logging_hook])

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\深圳\大岩\项目\intern-2018\CNN\estimator_mnist\model.ckpt-2001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 2001 into D:\深圳\大岩\项目\intern-2018\CNN\estimator_mnist\model.ckpt.
INFO:tensorflow:probabilities = [[0.00141846 0.02074195 0.00748468 0.6638555  0.00179395 0.12275656
  0.00220401 0.06959698 0.05794057 0.0522074 ]
 [0.00383784 0.23427759 0.01475341 0.08921957 0.00710426 0.07496464
  0.00681277 0.01693185 0.517067   0.03503102]
 [0.0023108  0.00004749 0.00615296 0.00123137

INFO:tensorflow:loss = 0.83387655, step = 2002
INFO:tensorflow:Saving checkpoints for 2002 into D:\深圳\大岩\项目\intern-2018\CNN\estimator_mnist\model.ckpt.
INFO:tensorflow:Loss for final step: 0.83387655.


In [8]:
mnist_classifier.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\深圳\大岩\项目\intern-2018\CNN\estimator_mnist\model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into D:\深圳\大岩\项目\intern-2018\CNN\estimator_mnist\model.ckpt.
INFO:tensorflow:loss = 2.3111615, step = 2
INFO:tensorflow:global_step/sec: 5.73513
INFO:tensorflow:loss = 2.2944481, step = 102 (17.436 sec)
INFO:tensorflow:global_step/sec: 5.84841
INFO:tensorflow:loss = 2.2911313, step = 202 (17.101 sec)
INFO:tensorflow:global_step/sec: 6.01187
INFO:tensorflow:loss = 2.260814, step = 302 (16.647 sec)
INFO:tensorflow:global_step/sec: 5.82767
INFO:tensorflow:loss = 2.2373242, step = 402 (17.146 sec)
INFO:tensorflow:global_step/sec: 5.62465
INFO:tensorflow:loss = 2.2413347, step = 502 (17.778 sec)
INFO:tensorflow:global_st

In [13]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-27-06:20:10
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\深圳\大岩\项目\intern-2018\CNN\estimator_mnist\model.ckpt-2001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-27-06:20:15
INFO:tensorflow:Saving dict for global step 2001: accuracy = 0.8354, global_step = 2001, loss = 0.70055634
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2001: D:\深圳\大岩\项目\intern-2018\CNN\estimator_mnist\model.ckpt-2001
{'accuracy': 0.8354, 'loss': 0.70055634, 'global_step': 2001}


tensorboard默认写入了一些scalar，但需要手动调整的怎么修改？还是说计算过的scalar就自动会存下来。